## Import Libraries
In this cell, we import **TensorFlow**, which is the primary library used here to build, train, and evaluate the deep learning model.

In [1]:
import tensorflow as tf

c:\Users\zakar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## Load the Dataset
Here we load the **Fashion MNIST** dataset from Keras.
* The data is unpacked into training and testing sets (images and labels).
* We also print the shape of `training_data` to confirm it consists of 60,000 images, each with dimensions of 28x28 pixels.

In [2]:
fmnist=tf.keras.datasets.fashion_mnist
(training_data, training_labels),(test_data, test_labels)=fmnist.load_data()
print(training_data.shape)

(60000, 28, 28)


## Normalize Data
The pixel values of the images originally range from 0 to 255.
* We divide the training and testing images by **255.0** to scale the values between **0 and 1**.
* Normalizing the data helps the neural network learn more efficiently.

In [3]:
training_data=training_data/255.0
test_data=test_data/255.0

## Define Custom Callback
We define a custom class `mycallback` that inherits from `tf.keras.callbacks.Callback`.
* **Purpose:** To monitor the accuracy at the end of every epoch.
* **Action:** If the accuracy reaches **95% (0.95)** or higher, the callback prints a message and cancels the training (`stop_training=True`). This saves time and prevents overfitting.

In [4]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs["accuracy"] >=0.95:
            print(f"the accuracy is more than or equal 0.95 so stop training")
            self.model.stop_training=True

## Define Neural Network Architecture

We define the layers of the Sequential model:

* **Input & Flatten:** Takes the 28x28 images and flattens them into a 1D vector.
* **Dense Layers (Hidden):** We add three hidden layers with **256**, **128**, and **32** neurons respectively. They use the `relu` (Rectified Linear Unit) activation function to learn complex patterns.
* **Output Layer:** The final Dense layer has **10 neurons** (corresponding to the 10 classes of clothing) and uses `softmax` activation to output a probability distribution for each class.

## Compile the Model
Before training, we configure the learning process:
* **Optimizer:** We use `Adam`, which is an efficient algorithm for adjusting the weights.
* **Loss Function:** We use `sparse_categorical_crossentropy` because our targets (labels) are integers and we are doing multi-class classification.
* **Metrics:** We want to track `accuracy` to see how often the model classifies images correctly.

In [5]:

model=tf.keras.models.Sequential([
    
    tf.keras.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])



model.compile(
    optimizer=tf.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

## Train the Model
We start the training loop using `model.fit`:
* **Data:** We feed in `training_data` and `training_labels`.
* **Epochs:** We set the maximum number of epochs to **50**.
* **Callbacks:** We pass our custom `mycallback()` instance.
    * *Note:* Even though we asked for 50 epochs, the callback will interrupt and stop the training automatically as soon as the accuracy hits **95%**, preventing unnecessary computation.

In [6]:

model.fit(training_data, training_labels, epochs=50, callbacks=[mycallback()])

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8249 - loss: 0.4894
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8667 - loss: 0.3626
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8783 - loss: 0.3281
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8869 - loss: 0.3048
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8942 - loss: 0.2848
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8983 - loss: 0.2705
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9020 - loss: 0.2609
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9056 - loss: 0.2516
Epoch 9/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9094 - loss: 0.2421
Epoch 10/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9125 - loss: 0.2325
Epoch 11/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9148 - loss: 0.2225
Epoch 12/50
1875/1875 ━━━

## Evaluate the Model
After training, we verify the model's performance on the **unseen test dataset**:
* `model.evaluate()` computes the loss and accuracy on the test data.
* We print the **Test Loss** and **Test Accuracy** to see how well the model generalizes to new images.

In [8]:
test_loss, test_accuracy=model.evaluate(test_data, test_labels, verbose=0)
print(f"The test loss is {test_loss}")
print(f"The test accuracy is equal {test_accuracy}")

The test loss is 0.4174339771270752
The test accuracy is equal 0.8928999900817871


## Analysis of Training vs. Testing Results

### 1. Training Performance
* **Early Stopping:** The model successfully reached the target accuracy of **95%** at Epoch 31, triggering our custom callback to stop training.
* **Final Training Metrics:**
    * Accuracy: ~95.16%
    * Loss: ~0.122

### 2. Testing Performance (Unseen Data)
* **Metrics:**
    * Test Accuracy: ~89.30%
    * Test Loss: ~0.417

### 3. Conclusion: Overfitting (High Variance)
By comparing the training and testing results, we observe a classic case of **Overfitting**:
* **The Gap:** There is a noticeable gap between training accuracy (95%) and test accuracy (89%).
* **Loss Discrepancy:** The test loss (0.417) is nearly **4x higher** than the training loss (0.122).
* **Meaning:** The model has learned the specific details and noise of the training data too well ("memorization") but struggles to generalize that knowledge effectively to new images it hasn't seen before.